# Cafe Feedback Exploration & Walkthrough

This notebook captures the thinking, data processing steps, and exploratory visuals that underpin the Streamlit dashboard. It consolidates the short walkthrough, an explanation of the process, and a quick description of data cleaning so the dashboard can stay focused on the AI analysis.

## Walkthrough & Process
- **Goal**: surface hard (ratings, spend) and soft (comments) signals for cafe locations, and deliver actionable insights.
- **Steps taken**: inspect raw schema → normalise `Location` strings → parse/clean ratings and currency → coerce timestamps → derive calendar fields → profile distributions for each column → build visuals.
- **Why this matters**: the raw file contains stray spaces in locations and timestamp-like strings in `Transaction Value`; without cleaning, filters and spend averages become misleading. Keeping cleaning documented here keeps the dashboard uncluttered.
- **How to refresh**: rerun this notebook on new drops; cleaned outputs and charts will stay in sync with the app.

## Quick explanation of data processing
- Drop empty `Unnamed` columns; coerce `Rating` to numeric.
- Strip/collapse whitespace in `Location` to merge duplicates (e.g., `Albany ` → `Albany`).
- Parse `Transaction Value` as currency, strip `$`/commas, and discard impossible spends (≤0 or >$500) to remove timestamp artefacts.
- Parse `Transaction Date and Time` as day-first datetime; derive `Date` and `DayName` for trend views.
- Persist these steps so dashboard filters and aggregates stay realistic.

In [ ]:
import re
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_theme()

DATA_FILE = Path("Sample data - Cafe - Sample data 2400 records.csv")
raw_df = pd.read_csv(DATA_FILE)
raw_df.head()

In [ ]:
# Cleaning helpers (mirrors dashboard logic)
def parse_transaction(val):
    if pd.isna(val):
        return None
    s = str(val).strip()
    m = re.search(r"-?\$?\s*([0-9]{1,3}(?:,[0-9]{3})*|[0-9]+)(?:\.[0-9]{1,2})?", s)
    if not m:
        return None
    try:
        v = float(re.sub(r"[,$]", "", m.group()))
    except ValueError:
        return None
    if v <= 0 or v > 500:
        return None
    return v

df = raw_df.copy()
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df["Location"] = df["Location"].astype(str).str.strip().str.replace(r"\s+", " ", regex=True)
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")
df["Transaction Value"] = df["Transaction Value"].apply(parse_transaction)
df["Transaction Date and Time"] = pd.to_datetime(df["Transaction Date and Time"], dayfirst=True, errors="coerce")
df["Date"] = df["Transaction Date and Time"].dt.date
df["DayName"] = df["Transaction Date and Time"].dt.day_name()
df.dropna(subset=["Rating", "Transaction Value"], inplace=True)
df.head()

## Categorical distributions (all categories printed)
For each categorical-like column, the full value counts are displayed. Plots are shown when the category count is manageable.

In [ ]:
categorical_cols = []
numeric_cols = []

for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        numeric_cols.append(col)
    elif pd.api.types.is_datetime64_any_dtype(df[col]):
        categorical_cols.append(col)
    else:
        categorical_cols.append(col)

for col in categorical_cols:
    print(f"\n==== {col} (categorical-like) ====")
    vc = df[col].fillna("<missing>").value_counts()
    display(vc)
    if 1 < len(vc) <= 30:
        vc.sort_values().plot(kind="barh", title=f"{col} frequency", figsize=(8, max(4, len(vc) * 0.25)))
        plt.xlabel("Count")
        plt.tight_layout()
        plt.show()

## Numeric distributions
Descriptive stats and histograms for each numeric-like column.

In [ ]:
for col in numeric_cols:
    print(f"\n==== {col} (numeric-like) ====")
    series = df[col].dropna()
    display(series.describe())
    series.plot(kind="hist", bins=20, title=f"{col} histogram", figsize=(6,4))
    plt.xlabel(col)
    plt.tight_layout()
    plt.show()

## Targeted visuals
- Rating distribution and average transaction by rating
- Daily trends of rating and spend (to mirror the dashboard)
- Top comment keywords

In [ ]:
rating_counts = df["Rating"].value_counts().sort_index()
rating_counts.plot(kind="bar", title="Rating distribution", ylabel="Count", xlabel="Rating")
plt.tight_layout()
plt.show()

avg_txn_by_rating = df.groupby("Rating")["Transaction Value"].mean()
avg_txn_by_rating.plot(kind="bar", title="Avg transaction by rating", ylabel="$", xlabel="Rating")
plt.tight_layout()
plt.show()

daily = (
    df.groupby("Date")
    .agg(avg_rating=("Rating", "mean"), avg_txn=("Transaction Value", "mean"))
    .sort_index()
)
daily.plot(y=["avg_rating", "avg_txn"], title="Daily trends: rating and spend", marker="o")
plt.tight_layout()
plt.show()

stop_words = {"the","and","to","of","a","in","for","with","is","it","on","my","our","at","are","was","be","have","has","that","they","this","i","we","you","their","as","so","its","by","from","an","were","your","also","us","had"}
word_counts = {}
for c in df["Comment"].dropna().astype(str):
    for w in re.findall(r"[a-zA-Z']+", c.lower()):
        if w not in stop_words and len(w) > 1:
            word_counts[w] = word_counts.get(w, 0) + 1
top_words = sorted(word_counts.items(), key=lambda kv: kv[1], reverse=True)[:12]
pd.DataFrame(top_words, columns=["word", "count"]).set_index("word").plot(kind="bar", title="Top words in comments")
plt.tight_layout()
plt.show()

## Key takeaways
- Location labels need normalisation; otherwise filters show duplicates.
- Cleaning transaction values removes timestamp artefacts and keeps spend realistic (~$15 median).
- Ratings skew high (4–5) while low ratings are scarce; focus on consistency and speed where comments flag issues.
- Daily trend views help spot emerging changes faster than monthly rolls.